In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime as dt
from scipy.spatial import distance
import scipy.stats as stats

%matplotlib inline

In [2]:
TIME_SAMPLE_FREQ = '60s'

In [3]:
df = pd.read_csv(".\\_events\\_generated\\valid_user_3\\base_bt_0.data", sep=';', index_col = False, header = None, low_memory = False, \
                 names = ['timestamp', 'action', 'bssid', 'major_class', 'class', \
                          'bond_state', 'type', 'user'])

In [4]:
df.head()

,timestamp,action,bssid,major_class,class,bond_state,type,user
0,2021-03-13 20:12:13.128,android.bluetooth.device.action.FOUND,59:2D:94:4A:2D:D9,7936,7936,10,2,3
1,2021-03-13 20:12:14.074,android.bluetooth.device.action.FOUND,EC:3C:BB:1E:E6:8B,512,524,10,1,3
2,2021-03-13 20:12:14.124,android.bluetooth.device.action.FOUND,EC:3C:BB:1E:E6:8B,512,524,10,1,3
3,2021-03-13 20:12:14.415,android.bluetooth.device.action.FOUND,40:34:C6:14:A9:58,7936,7936,10,2,3
4,2021-03-13 20:12:14.482,android.bluetooth.device.action.FOUND,4B:91:A2:D9:FF:D7,7936,7936,10,2,3


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 8 columns):
timestamp      263 non-null object
action         263 non-null object
bssid          263 non-null object
major_class    263 non-null int64
class          263 non-null int64
bond_state     263 non-null int64
type           263 non-null int64
user           263 non-null int64
dtypes: int64(5), object(3)
memory usage: 16.5+ KB


In [6]:
df = df[df['action'] == 'android.bluetooth.device.action.FOUND']

In [7]:
df.index = pd.DatetimeIndex(df.timestamp)
df = df.sort_index()

In [8]:
VALID_USER = df.iloc[0]['user']
df['events_count'] = 1

In [9]:
df = df.drop(['timestamp', 'action', 'class', 'major_class', 'bond_state', 'type'], axis = 1)

In [10]:
bssid_map = { bssid.replace(' ', ''): idx for bssid, idx in zip(df.bssid.unique(), range(len(df.bssid.unique()))) }

In [11]:
df.bssid = df.bssid.apply(lambda x: str(x).replace(' ', ''))

In [12]:
df['count'] = 1

In [13]:
def agg_string_join(col):
    col = col.apply(lambda x: str(x))
    return col.str.cat(sep = ',').replace(' ', '')

In [14]:
def agg_bssid_col(col):
    array_len = len(bssid_map)
    array = np.zeros(array_len, dtype = 'int8')
    def fill_array(bssid):
        array[bssid_map[bssid.replace(' ', '')]] = 1
        return
        
    col.apply(lambda x: fill_array(x))
    return np.array2string(array, separator = ',').replace(' ', '')[1:-1]

In [15]:
one_hot_columns_count = 0
for col in df.columns:
    if col.find('one_hot') != -1:
        one_hot_columns_count += 1

In [16]:
def user_agg(col):
    if (col == VALID_USER).all():
        return 1
    else:
        return 0

In [17]:
cat_columns = df.columns[1:1 + one_hot_columns_count]
cat_columns_map = { col: 'mean' for col in cat_columns }

all_func_dicts_quantum = { 'bssid' : agg_bssid_col, 'count' : 'sum', 'user' : user_agg, 'events_count' : 'sum' }
all_func_dicts_quantum.update(cat_columns_map)

In [18]:
df_quantum = df.groupby(pd.Grouper(freq = '5s'), as_index=True).agg(all_func_dicts_quantum)

In [19]:
df_quantum = df_quantum.reset_index()
df_quantum.index = pd.DatetimeIndex(df_quantum.timestamp)

In [20]:
df_quantum = df_quantum.dropna()

In [21]:
df_le = pd.read_csv(".\\_events\\_generated\\valid_user_3\\le_bt_3.data", sep = ';', index_col = False, header = None, low_memory = False, \
                 names = ['timestamp', '1', '2', '3', 'level', '3', 'connectable', '4'])

df_le = df_le.drop(df_le.columns.difference(['connectable','timestamp', 'level']), axis = 1)
df_le.index = pd.DatetimeIndex(df_le.timestamp)
df_le = df_le.sort_index()

D:\Program Files\Anaconda3\lib\site-packages\pandas\io\parsers.py:702: UserWarning: Duplicate names specified. This will raise an error in the future.
  return _read(filepath_or_buffer, kwds)


In [22]:
df_le['connectable'] = df_le['connectable'].apply(lambda x: 1 if str(x).lower() == 'true' else 0)

In [23]:
df_le = df_le.groupby(pd.Grouper(freq = '5s'), as_index=True).agg({'level':'mean', 'connectable':'mean'})

In [24]:
df_le = df_le.dropna()

In [25]:
def get_le_conn_status_from_row(row):
    conn = df_le.iloc[df_le.index.get_loc(row.name, method = 'nearest')]['connectable']
    time = df_le.iloc[df_le.index.get_loc(row.name, method = 'nearest')].name
    return conn if abs((time - row.name).total_seconds()) < 10 else 0

def get_le_level_from_row(row):
    level = df_le.iloc[df_le.index.get_loc(row.name, method = 'nearest')]['level']
    time = df_le.iloc[df_le.index.get_loc(row.name, method = 'nearest')].name
    return level if abs((time - row.name).total_seconds()) < 10 else 0


df_quantum['le_connectable'] = df_quantum.apply(lambda row: get_le_conn_status_from_row(row), axis = 1)
df_quantum['le_level'] = df_quantum.apply(lambda row: get_le_level_from_row(row), axis = 1)

In [26]:
def string2array(string):
    try:
        array = np.fromstring(string, sep=',')
        return array
    except:
        return np.nan

def to_ones_array(array):
    try:
        array[array != 0] = 1
        return array
    except:
        return np.nan

def get_len(obj):
    try:
        length = len(obj)
        return length
    except:
        return np.nan

In [27]:
def get_occured_nets_count(row, prev_col, curr_col):
    prev = to_ones_array(string2array(row[prev_col]))
    curr = to_ones_array(string2array(row[curr_col]))
    intersection = np.logical_and(curr, prev)
    diff = np.logical_and(curr, np.logical_not(intersection))
    
    if (np.count_nonzero(np.logical_or(prev, curr)) == 0):
        return 0
    
    return np.count_nonzero(diff) / np.count_nonzero(np.logical_or(prev, curr))

def get_disappeared_nets_count(row, prev_col, curr_col):
    prev = to_ones_array(string2array(row[prev_col]))
    curr = to_ones_array(string2array(row[curr_col]))
    intersection = np.logical_and(curr, prev)
    diff = np.logical_and(prev, np.logical_not(intersection))
    
    if (np.count_nonzero(np.logical_or(prev, curr)) == 0):
        return 0
    
    return np.count_nonzero(diff) / np.count_nonzero(np.logical_or(prev, curr))

def get_jaccard_index(row, prev_col, curr_col):
    prev = to_ones_array(string2array(row[prev_col]))
    curr = to_ones_array(string2array(row[curr_col]))
    return distance.jaccard(prev, curr)

def get_occur_speed(row, prev_col, curr_col):
    prev = to_ones_array(string2array(row[prev_col]))
    curr = to_ones_array(string2array(row[curr_col]))
    return np.linalg.norm(prev - curr) / np.sqrt(get_len(prev))

def calc_single_cols_in_window(df, col, new_col, window, func):
    def func_wrapper(func, row, prev_col, curr_col):
        delta = row.timestamp - row.prev_timestamp
        if pd.isnull(delta):
            delta = 0
        else:
            delta = abs(delta.total_seconds())
        if delta > 10 * 60:
            return np.nan
        else:
            return func(row, prev_col_name, col)
        
    new_cols = []
        
    for i in range(window):
        prev_col_name = "_".join(['prev', col, str(i + 1)])
        new_col_name = "_".join([new_col, str(i + 1)])
        
        df.loc[:, 'prev_timestamp'] = df.timestamp.shift(i + 1)
        df.loc[:, prev_col_name] = df[col].shift(i + 1)
        df.loc[:, new_col_name] = df.apply(lambda row: func_wrapper(func, row, prev_col_name, col), axis = 1)
        df = df.drop(prev_col_name, axis = 1)
        df = df.drop('prev_timestamp', axis = 1)
        new_cols.append(new_col_name)
        
    df.loc[:, "_".join([new_col, 'mean'])] = df[new_cols].mean(axis = 1)
    df.loc[:, "_".join([new_col, 'median'])] = df[new_cols].median(axis = 1)
    df.loc[:, "_".join([new_col, 'var'])] = df[new_cols].var(axis = 1)
        
    return df

In [28]:
WINDOW_SIZE = 5

occur_and_level_columns_map = [
    ("bssid", "occured_devices_count", WINDOW_SIZE, get_occured_nets_count),
    ("bssid", "disappeared_devices_count", WINDOW_SIZE, get_disappeared_nets_count),
    ("bssid", "jaccard_index", WINDOW_SIZE, get_jaccard_index), 
    ("bssid", "occur_speed", WINDOW_SIZE, get_occur_speed)
]

for (col, new_col, window, func) in occur_and_level_columns_map:
    df_quantum = calc_single_cols_in_window(df_quantum, col, new_col, window, func)

In [29]:
def get_conn_level_speed(row, prev_col, curr_col):
    return row[curr_col] - row[prev_col]

In [30]:
single_columns_map = [
    ("count", "count_speed", WINDOW_SIZE, get_conn_level_speed)
]

for (col, new_col, window, func) in single_columns_map:
    df_quantum = calc_single_cols_in_window(df_quantum, col, new_col, window, func)

In [31]:
def agg_str(col):
    all_freq = col.str.cat(sep=',')
    return string2array(all_freq)

def str_mean(col):
    return np.mean(agg_str(col))

def str_var(col):
    return np.var(agg_str(col))

def str_median(col):
    return np.median(agg_str(col))

def str_skew(col):
    return stats.skew(agg_str(col))

def str_kurt(col):
    return stats.kurtosis(agg_str(col))

def mean(col):
    return np.mean(col)

def var(col):
    return np.var(col)

def median(col):
    return np.median(col)

def skew(col):
    return stats.skew(col)

def kurt(col):
    return stats.kurtosis(col)

In [32]:
cols_for_drop = []
names = [
    "occured_devices_count",
    "disappeared_devices_count",
    "jaccard_index",
    "occur_speed",
    "count_speed"
]

for i in range(1, WINDOW_SIZE + 1):
    for name in names:
        cols_for_drop.append('_'.join([name, str(i)]))
        
df_quantum = df_quantum.drop(['bssid', 'timestamp'], axis = 1)
df_quantum = df_quantum.drop(cols_for_drop, axis = 1)

In [33]:
df_quantum.columns

Index(['count', 'user', 'events_count', 'le_connectable', 'le_level',
       'occured_devices_count_mean', 'occured_devices_count_median',
       'occured_devices_count_var', 'disappeared_devices_count_mean',
       'disappeared_devices_count_median', 'disappeared_devices_count_var',
       'jaccard_index_mean', 'jaccard_index_median', 'jaccard_index_var',
       'occur_speed_mean', 'occur_speed_median', 'occur_speed_var',
       'count_speed_mean', 'count_speed_median', 'count_speed_var'],
      dtype='object')

In [34]:
def user_agg(col):
    if (col == 1).all():
        return 1
    else:
        return 0

In [35]:
common_cols = [x for x in df_quantum.columns[:one_hot_columns_count + 3] if x != 'user' and x != 'events_count']
speed_acc_cols = df_quantum.columns[one_hot_columns_count + 3:]

common_funcs_list = [mean, var, median, skew, kurt]
special_funcs_list = [mean, pd.DataFrame.mad, skew]

common_cols_map = { col : common_funcs_list for col in common_cols }
speed_acc_cols_map = { col : special_funcs_list for col in speed_acc_cols }

additional = { 'user' : user_agg, 'events_count' : 'sum' }

agg_dict = common_cols_map
agg_dict.update(speed_acc_cols_map)
agg_dict.update(additional)

In [36]:
df_quantum[speed_acc_cols] = df_quantum[speed_acc_cols].apply(pd.to_numeric)

In [37]:
df_sampling = df_quantum.groupby(pd.Grouper(freq = TIME_SAMPLE_FREQ)).agg(agg_dict)

In [38]:
df_rolling = df_quantum.rolling(TIME_SAMPLE_FREQ, min_periods = 1, center = False).agg(agg_dict)

In [39]:
df_sampling.columns = ["_".join([str(high_level_name), str(low_level_name)]) \
                       for (high_level_name, low_level_name) in df_sampling.columns.values]

df_rolling.columns = ["_".join([str(high_level_name), str(low_level_name)]) \
                      for (high_level_name, low_level_name) in df_rolling.columns.values]

In [40]:
(df_quantum['user'] == 1).all()

False

In [41]:
df_sampling = df_sampling.dropna()
df_sampling = df_sampling.fillna(0)

df_rolling = df_rolling.dropna()
df_rolling = df_rolling.fillna(0)

In [42]:
df_sampling.to_csv(".\\_datasets\\60s\\bt_sampling_dataset_3.csv")
df_rolling.to_csv(".\\_datasets\\60s\\bt_rolling_dataset_3.csv")

FileNotFoundError: [Errno 2] No such file or directory: '.\\_datasets\\60s\\bt_sampling_dataset_3.csv'